In [1]:
from pulp import LpMaximize, LpProblem, LpVariable, LpMinimize, LpMaximize, lpSum, PULP_CBC_CMD, GLPK_CMD

In [5]:
# Simple integer programming problem - testing PulP's accuracy

# Company manufactures sporks (spoon-fork-knife combo), packets and school packs. 
# Packets are sets of a spork, a napkin and a straw.
# School packs are boxes of 100 packs with additional 10 sporks.
#
# For 1000 sporks, it takes :
# 1. 0.8 hours of molding machine time.
# 2. 0.2 hours of supervisory time
# 3. $2.50 in direct costs.
#
# For 1000 packets with 1 spork, 1 napkin and 1 straw, it takes :
# 1. 1.5 hours of packaging capacity.
# 2. 0.5 hours of supervisory time.
# 3. $4.00 in direct cost.
#
# For 1000 school packs:
# 2.5 hours of packaging capacity.
# 0.5 hours of supervisory time.
# 10 sporks, 100 packets and $8.00 direct costs.

# Any of the three products may be sold in unlimited quantities at prices of $5.00, $15.00, and $300.00 per
# thousand, respectively. If there are 200 hours of production time in the coming month, what products, and
# how much of each, should be manufactured to yield the most profit?

model = LpProblem(name="simple-mixed-integer-prob", sense=LpMaximize)

# Define variables
# x1 = thousands of sporks produced
# x2 = housands of packets  produced
# x3 = thousands of school packs produced

# y1 = x1 - x2 - 10*x3    # 1 packet needs 1 spork and 1 school pack needs 10 sporks
# y2 = x2 - 100*x3        # 100 packets in school packs 
# y3 = x3

# Express x's in terms of y's
# x1 = y1 + y2 + 110*y3
# x2 = y2 + 100*y3
# x3 = y3

# Total profit = 5.00 * y1 + 15.00 * y2 + 300.00 * y3 - (2.5 * x1 + 4.00 * x2 + 8.00 * x3)
# In terms of x's only, Total Profits = 2.5 * x1 + 6 * x2 -1258 * x3

# Define variables
x1 = LpVariable(name="sporks_prod", lowBound=0, cat="Integer")           # thousands of sporks produced
x2 = LpVariable(name="packets_prod", lowBound=0, cat="Integer")          # thousands of packets  produced
x3 = LpVariable(name="school_packs_prod", lowBound=0, cat="Integer")     # thousands of school packs produced


# Objective function
model += 2.5 * x1 + 6 * x2 -1258 * x3, "Profit"

# Constraints due to 200 hours limit for 1 month 
model += 0.8 * x1 <= 200, "Molding"                                 # Sporks requires 0.8 hours of molding per thousand
model += 1.5 * x2 + 2.5 * x3 <= 200, "Packaging"                    # Packets and School packs require 1.5 and 2.5 hours of packaging per thousand
model += 0.2 * x1 + 0.5 * x2 + 0.5 * x3 <= 200, "Supervisory"       # Supervisory hours required for all per thousand

# Solve the problem
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Print results
print("Results from using CBC solver :")
print(f"Optimal Production of A: {x1.varValue}")
print(f"Optimal Production of B: {x2.varValue}")
print(f"Optimal Production of C: {x3.varValue}")
print(f"Maximum Profit: {model.objective.value()}")
print(f"Maximum Profit (Manual calcs): {2.5 * x1.varValue + 6 * x2.varValue -1258 * x3.varValue}")


Results from using CBC solver :
Optimal Production of A: 250.0
Optimal Production of B: 133.0
Optimal Production of C: 0.0
Maximum Profit: 1423.0
Maximum Profit (Manual calcs): 1423.0


In [25]:
# Assign truck using integer programming - using direct truck assigment - i.e. each truck is selected using a binary decision variable 
# Major disadvantages - Assigns only 1 truck per time step (such as at a dump event), even if a shovel could use more than 1 for the next planning period ( which need to be estimated)
# **Note** This example does not represent a realistic fleet configuration, too few trucks, so only for showing concept.

# Define the number of trucks and shovels
m = 6  # Number of trucks
n = 4  # Number of shovels

# Create the decision variables X_ij (binary)
X = [[LpVariable(f"X_{i}_{j}", cat="Binary") for j in range(n)] for i in range(m)]

# Define expected travel time parameters W_ij as fixed constants (in minutes)
# Could add exepected truck queue time when truck is expected to arrive at nominated shovel
W = [[18, 15, 20, 2],   # Idle time for truck 1 at different shovels
     [18, 45, 18, 32],  # Idle time for truck 2 at different shovels
     [17, 14, 25, 52],  # Idle time for truck 3 at different shovels
     [19, 28, 19, 21],  # Idle time for truck 4 at different shovels
     [16, 10, 17, 33],  # Idle time for truck 5 at different shovels
     [19, 10, 17, 29]]  

# Define the optimization problem
model = LpProblem("direct_truck_assignment_by_travel_minimization", LpMinimize)

# Define the objective function: Minimize total travel time
model += lpSum(W[i][j] * X[i][j] for i in range(m) for j in range(n))

# Constraints (example: each truck is can be assigned to at most one shovel)
for i in range(m):
    model += lpSum(X[i][j] for j in range(n)) <= 1  # Each truck assigned to at most one shovel


for j in range(n):
    model += lpSum(X[i][j] for i in range(m)) == 1 # Each shovel is assigned exactly 1 truck

# Solve the problem
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Print results
for i in range(m):
    for j in range(n):
        if X[i][j].varValue == 1:
            print(f"Truck {i+1}, Shovel {j+1}: Assigned = {X[i][j].varValue}, Travel Time = {W[i][j]}")

print(f"Minimum total travelling time = {model.objective.value()} mins")        


Truck 1, Shovel 4: Assigned = 1.0, Travel Time = 2
Truck 2, Shovel 3: Assigned = 1.0, Travel Time = 18
Truck 5, Shovel 1: Assigned = 1.0, Travel Time = 16
Truck 6, Shovel 2: Assigned = 1.0, Travel Time = 10
Minimum total travelling time = 46.0 mins


In [24]:
X

[[X_0_0, X_0_1, X_0_2],
 [X_1_0, X_1_1, X_1_2],
 [X_2_0, X_2_1, X_2_2],
 [X_3_0, X_3_1, X_3_2],
 [X_4_0, X_4_1, X_4_2],
 [X_5_0, X_5_1, X_5_2]]

$$
\begin{array}{l l}
\text{General tranportation (Temeng) optimization of material flow rates from shovels to dumps, crushers, stockpiles.} \\
\text{In this basic formulation, there is no distiguishing between dumps, crushers or stockpiles, only dumps.} \\
\quad \\
\text{m - number of supply points - such as shovels}  \\
\quad \\
\text{n - number of  demand points -  such as dumps} \\
\quad \\
\text{max/min} \quad \sum_{i=1}^{n} \sum_{j=1}^{m} C_{ij} X_{ij} \\
\quad \\
\text{subject to,} \\
\quad \\
\sum_{j=1}^{m} X_{ij} \leq s_{i} & \text{i=1,..,n} & \quad \text{Supply constraints }\\ 
\quad \\
\sum_{i=1}^{n} X_{ij} \geq d_{i} & \text{j=1,..,m} & \quad \text{Demand constraints }\\ 
\quad \\
X_{ij} \geq 0 &  \text{i=1,..,n} & \quad \text{j=1,..,m}\\
\quad \\
X_{ij} \geq 0 \\ & \text{Supply is greater or equal to demand}
\quad \\
\sum_{i=1}^{m} s_{i} \geq \sum_{i=1}^{n} d_{i} \\
\quad\\
X_{ij} - \text{number of units shipped from supply point i to demand point j }
\end{array}
$$

In [ ]:
# Implementation of transportation model
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, GLPK_CMD

# Define the number of trucks and shovels
m = 6  # Number of mining faces assigned to shovels - these are the sources of material   
n = 3  # Number of dumping points # crushers, stockpiles, waste dumps - these are the sinks of materials

# Create the decision variables X_ij (binary)
X = [[LpVariable(f"X_{i}_{j}", cat="Continuous") for j in range(n)] for i in range(m)]  # Tons of material transported from 

# Define cost parameters W_ij as fixed constants (example values). 
# Use travelling duration as the cost
# Each row is one of m sources and each column is one of n sinks 
# Each element in the matrix is the travelling duration (minutes) between a source and a sink        
C = [[18, 15, 20],    # Traveling time from shovel 1 to each of n dumping points
     [18, 40, 18],    # Traveling time from shovel 2 to each of n dumping points
     [17, 14, 25],    # Traveling time from shovel 3 to each of n dumping points
     [19, 28, 19],    # Traveling time from shovel 4 to each of n dumping points
     [16, 10, 17],    # Traveling time from shovel 5 to each of n dumping points
     [19, 10, 17]]    # Traveling time from shovel 6 to each of n dumping points

# Define the optimization problem
model = LpProblem("Load_to_Dump_flow_rates", LpMinimize)

# Define the objective function: Minimize total idle time
model += lpSum(W[i][j] * X[i][j] for i in range(m) for j in range(n))

# Constraints (example: each truck is can be assigned to at most one shovel)
for i in range(m):
    model += lpSum(X[i][j] for j in range(n)) <= 1  # Each truck assigned to at most one shovel


for j in range(n):
    model += lpSum(X[i][j] for i in range(m)) == 1 # Each shovel is assigned exactly 1 truck

# Solve the problem
model.solve()

# Print results
for i in range(m):
    for j in range(n):
        if X[i][j].varValue == 1:
            print(f"Truck {i+1}, Shovel {j+1}: Assigned = {X[i][j].varValue}, Wait Time = {W[i][j]}")

print(f"Minimum total idle time = {model.objective.value()} secs")        


Minimum total idle time = 0.0 secs


In [12]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2']
dumps = ['Dump1', 'Dump2']

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 200, 'Shovel2': 250}

# Dumping location capacity (tons per hour)
cap = {'Dump1': 180, 'Dump2': 270}

# Travel duration cost (minutes per ton)
travel_time = {
    ('Shovel1', 'Dump1'): 25,
    ('Shovel1', 'Dump2'): 28,
    ('Shovel2', 'Dump1'): 27,
    ('Shovel2', 'Dump2'): 26
}

# Shift duration (hours)
T_shift = 8

# Total hauled tonnage for the shift
total_tonnage = 3000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in dumps }

# Objective function: Minimize total travel duration
model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in dumps)

# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in dumps) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in dumps:
    model += lpSum(x[i, j] for i in shovels) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in dumps) * T_shift == total_tonnage, "TotalTonnageConstraint"

# Solve the model
model.solve()

# Print results
for i in shovels:
    for j in dumps:
        print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")

print(f"Total transportation cost: {model.objective.value()}")


Material transported from Shovel1 to Dump1: 180.0 tons per hour
Material transported from Shovel1 to Dump2: 0.0 tons per hour
Material transported from Shovel2 to Dump1: 0.0 tons per hour
Material transported from Shovel2 to Dump2: 195.0 tons per hour
Total transportation cost: 9570.0


In [13]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
from pulp import PULP_CBC_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 900, 'Shovel2': 950, 'Shovel3': 720, 'Shovel4': 880, 'Shovel5': 610, 'Shovel6': 1030, 'Shovel7': 460}  # Shovel7 mines waste

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (minutes per ton)
travel_time    = {
    ('Shovel1', 'Crusher1'): 50, ('Shovel1', 'Crusher2'): 70, ('Shovel1', 'Stockpile1'): 90, ('Shovel1', 'Stockpile2'): 100, ('Shovel1', 'Stockpile3'): 120,
    ('Shovel2', 'Crusher1'): 60, ('Shovel2', 'Crusher2'): 80, ('Shovel2', 'Stockpile1'): 110, ('Shovel2', 'Stockpile2'): 90, ('Shovel2', 'Stockpile3'): 140,
    ('Shovel3', 'Crusher1'): 70, ('Shovel3', 'Crusher2'): 60, ('Shovel3', 'Stockpile1'): 100, ('Shovel3', 'Stockpile2'): 130, ('Shovel3', 'Stockpile3'): 150,
    ('Shovel4', 'Crusher1'): 60, ('Shovel4', 'Crusher2'): 90, ('Shovel4', 'Stockpile1'): 120, ('Shovel4', 'Stockpile2'): 110, ('Shovel4', 'Stockpile3'): 160,
    ('Shovel5', 'Crusher1'): 50, ('Shovel5', 'Crusher2'): 70, ('Shovel5', 'Stockpile1'): 90, ('Shovel5', 'Stockpile2'): 100, ('Shovel5', 'Stockpile3'): 120,
    ('Shovel6', 'Crusher1'): 60, ('Shovel6', 'Crusher2'): 80, ('Shovel6', 'Stockpile1'): 110, ('Shovel6', 'Stockpile2'): 90, ('Shovel6', 'Stockpile3'): 140,
    ('Shovel7', 'WasteDump'): 40  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 75000

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time }


# Objective function: Minimize total travel duration
#model += lpSum(travel_time[i, j] * x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time)
model += lpSum(travel_time[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time)


# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift == total_tonnage, "TotalTonnageConstraint"

# Waste material constraint: Any material from Shovel7 must only go to WasteDump
model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Crusher prioritization: Ensure crushers receive more tonnage before stockpiles
for j in stockpiles:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= lpSum(x[i, c] for i in shovels for c in crushers if (i, c) in travel_time), f"CrusherPriority_{j}"




# Additional constraints to prevent negative flows
# for i in shovels:
#     for j in all_dumps:
#         if (i, j) in travel_time:
#             model += x[i, j] >= 0, f"NonNegativeFlow_{i}_{j}"

#Instead of this model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift == total_tonnage
# Do this - model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage

model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Solve the model
#model.solve()

# Print results
for i in shovels:
    for j in all_dumps:
        if (i, j) in travel_time:
            print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")

print(f"Total transportation cost: {model.objective.value()}")


Material transported from Shovel1 to Crusher1: 0.0 tons per hour
Material transported from Shovel1 to Crusher2: 280.0 tons per hour
Material transported from Shovel1 to Stockpile1: 0.0 tons per hour
Material transported from Shovel1 to Stockpile2: 0.0 tons per hour
Material transported from Shovel1 to Stockpile3: 250.0 tons per hour
Material transported from Shovel2 to Crusher1: 0.0 tons per hour
Material transported from Shovel2 to Crusher2: 0.0 tons per hour
Material transported from Shovel2 to Stockpile1: 0.0 tons per hour
Material transported from Shovel2 to Stockpile2: 0.0 tons per hour
Material transported from Shovel2 to Stockpile3: 0.0 tons per hour
Material transported from Shovel3 to Crusher1: 0.0 tons per hour
Material transported from Shovel3 to Crusher2: 0.0 tons per hour
Material transported from Shovel3 to Stockpile1: 0.0 tons per hour
Material transported from Shovel3 to Stockpile2: 0.0 tons per hour
Material transported from Shovel3 to Stockpile3: 0.0 tons per hour
Mat

In [14]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

# Define sets of shovels and dumping locations
shovels = ['Shovel1', 'Shovel2', 'Shovel3', 'Shovel4', 'Shovel5', 'Shovel6', 'Shovel7']
crushers = ['Crusher1', 'Crusher2']
stockpiles = ['Stockpile1', 'Stockpile2', 'Stockpile3']
waste_dump = ['WasteDump']
all_dumps = crushers + stockpiles + waste_dump

# Productivity of each shovel (tons per hour)
prod = {'Shovel1': 12000, 'Shovel2': 250, 'Shovel3': 220, 'Shovel4': 180, 'Shovel5': 210, 'Shovel6': 230, 'Shovel7': 260}  # Shovel7 mines waste

# Dumping location capacity (tons per hour)
cap = {'Crusher1': 300, 'Crusher2': 280, 'Stockpile1': 200, 'Stockpile2': 220, 'Stockpile3': 250, 'WasteDump': 500}

# Travel duration cost (total minutes)
travel_time = {
    ('Shovel1', 'Crusher1'): 15, ('Shovel1', 'Crusher2'): 18, ('Shovel1', 'Stockpile1'): 25, ('Shovel1', 'Stockpile2'): 28, ('Shovel1', 'Stockpile3'): 30,
    ('Shovel2', 'Crusher1'): 17, ('Shovel2', 'Crusher2'): 20, ('Shovel2', 'Stockpile1'): 27, ('Shovel2', 'Stockpile2'): 29, ('Shovel2', 'Stockpile3'): 32,
    ('Shovel3', 'Crusher1'): 19, ('Shovel3', 'Crusher2'): 16, ('Shovel3', 'Stockpile1'): 26, ('Shovel3', 'Stockpile2'): 30, ('Shovel3', 'Stockpile3'): 34,
    ('Shovel4', 'Crusher1'): 18, ('Shovel4', 'Crusher2'): 22, ('Shovel4', 'Stockpile1'): 28, ('Shovel4', 'Stockpile2'): 30, ('Shovel4', 'Stockpile3'): 36,
    ('Shovel5', 'Crusher1'): 15, ('Shovel5', 'Crusher2'): 18, ('Shovel5', 'Stockpile1'): 25, ('Shovel5', 'Stockpile2'): 27, ('Shovel5', 'Stockpile3'): 30,
    ('Shovel6', 'Crusher1'): 17, ('Shovel6', 'Crusher2'): 20, ('Shovel6', 'Stockpile1'): 27, ('Shovel6', 'Stockpile2'): 29, ('Shovel6', 'Stockpile3'): 32,
    ('Shovel7', 'WasteDump'): 12  # Waste shovel only goes to WasteDump
}

# Shift duration (hours)
T_shift = 10

# Total hauled tonnage for the shift
total_tonnage = 60000  # Large-scale test case - each crusher at 2000 tons/hour for 10 hours shift = 40000 tons. Remainder goes to stock piles (2 iron ore)

# Define the optimization problem
model = LpProblem("Mining_Optimization", LpMinimize)

# Define decision variables
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time }

# Objective function: Minimize total travel duration + crusher balance penalty
balance_diff = LpVariable("BalanceDiff", lowBound=0)
model += lpSum(travel_time[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) + balance_diff

# Constraints
# Shovel production limits
for i in shovels:
    model += lpSum(x[i, j] for j in all_dumps if (i, j) in travel_time) <= prod[i], f"ShovelLimit_{i}"

# Dumping location capacity constraints
for j in all_dumps:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= cap[j], f"DumpCapacity_{j}"

# Total hauled tonnage constraint
model += lpSum(x[i, j] for i in shovels for j in all_dumps if (i, j) in travel_time) * T_shift <= total_tonnage, "TotalTonnageConstraint"

# Waste material constraint: Any material from Shovel7 must only go to WasteDump
model += lpSum(x['Shovel7', j] for j in all_dumps if (('Shovel7', j) in travel_time and j != 'WasteDump')) == 0, "WasteShovelConstraint"

# Crusher prioritization: Ensure crushers receive more ore before stockpiles
for j in stockpiles:
    model += lpSum(x[i, j] for i in shovels if (i, j) in travel_time) <= 0.1 * (cap["Crusher1"] + cap["Crusher2"]), f"CrusherPriority_{j}"

# Crushers must be balanced as evenly as possible
model += balance_diff >= lpSum(x[i, "Crusher1"] for i in shovels) - lpSum(x[i, "Crusher2"] for i in shovels)
model += balance_diff >= lpSum(x[i, "Crusher2"] for i in shovels) - lpSum(x[i, "Crusher1"] for i in shovels)

# Solve the model with PuLP CBC solver for better precision
model.solve(PULP_CBC_CMD(msg=True, options=['presolve', 'strong', 'numerical']))

# Print results
for i in shovels:
    for j in all_dumps:
        if (i, j) in travel_time:
            print(f"Material transported from {i} to {j}: {x[i, j].varValue} tons per hour")

print(f"Total transportation cost: {model.objective.value()}")


KeyError: ('Shovel7', 'Crusher1')

In [ ]:
x = { (i, j): LpVariable(f"x_{i}_{j}", lowBound=0) for i in shovels for j in all_dumps if (i, j) in travel_time }
x.keys(), x.values()

$$
\begin{array}{l l}
\text{Minimize idle times approach} \\
\quad \\
\text{minimize} \quad \sum_{i=1}^{T} \sum_{j=1}^{D} \sum_{j=1}^{S} C_{tds} X_{tds} + VBN(1-AF) \\
\quad \\
\text{subject to,} \\
\quad \\
\sum_{d=1}^{D} \sum_{s=1}^{S} tc_t * X_{tds} \leq TC_{t} & \text{t=1,..,T} & \quad \text{Truck t cannot accept loads more than its nominal capacity}\\ 
\quad \\
\sum_{t=1}^{T} \sum_{d=1}^{D} tc_t * X_{tds} \leq sc_{s} & \text{s=1,..,S} & \quad \text{ensures that summation of nominal capacity of all the trucks assigned to shovels does not exceed the shovel’s nominal digging rate (capacity)}\\ 
\quad \\
\sum_{t=1}^{T} \sum_{s=1}^{S} tc_t * X_{tds} \geq AF * pc_{d} & \text{d=1,..,D} \\ 
\quad \\
X_{tds} \in \text{\{}{0,1}\text{\}}&\quad\text{Binary decision varriables} \\
\quad \\
mf = \sum_{t=1}^{T} tc_t * X_{tds} / \sum_{s=1}^{S} \sum_{d=1}^{D} (pf_{sd} - pmsf_{sd}) \quad \\
\quad \\
C_{tds}= ltt_{td} + qt_{td} + dt_{td} + ett_{ts} - \sum_{tt=1}^{TT}(tinq_{tts} + tenr_{tts} x (st_{tts} + lt_{tts}) \\
\quad \\
where, \quad \\
\quad \\
X_{tds} \in \text{\{}{0,1}\text{\}}&\quad\text{Binary decision varriables} \\
\quad \\

\end{array}
$$

In [ ]:
# Crusher tons per hour estimate
tpy = 14e6 #tons per year
hours_per_year = 12 * 356
tph = tpy/hours_per_year
tph